# Задача 1. Введение в гетерогенную параллелизацию

    Объясните, что такое гетерогенная параллелизация.
        В ответе раскройте следующие аспекты:
        1. различия между параллельными вычислениями на CPU и GPU;
        2. преимущества гетерогенной параллелизации;
        3. примеры реальных приложений, в которых используется гетерогенная
        параллелизация.


### 1) Гетерогенная параллелизация

    Гетерогенная параллелизация — это подход к выполнению вычислительных задач, при котором одновременно используются разные типы вычислительных устройств в одной системе,
    например центральный процессор (CPU) и графический процессор (GPU).
    Основная цель — задействовать сильные стороны каждого устройства для повышения производительности и эффективности вычислений.

### 2) Различия между параллельными вычислениями на CPU и GPU

    CPU имеет относительно небольшое число мощных ядер, оптимизированных для сложной логики, ветвлений и последовательного выполнения инструкций. GPU содержит тысячи мелких ядер и предназначен для массово-параллельных операций над большими массивами данных.

    CPU: подходит для операций с интенсивной логикой, ветвлениями и динамическим потоком управления.

    GPU: эффективен при однотипных численных операциях, например в линейной алгебре, обработке изображений или обучении нейронных сетей.

    Использование только CPU или только GPU часто приводит к неоптимальному распределению ресурсов и снижению производительности.

### 3) Преимущества гетерогенной параллелизации

    1. Максимальная эффективность ресурсов — CPU выполняет управляющие задачи, а GPU обрабатывает массивные однотипные данные.

    2. Сокращение времени выполнения — распределение задач по устройствам ускоряет обработку больших объёмов данных.

    3. Гибкость и масштабируемость — задачи распределяются в зависимости от архитектуры устройств и объёма данных.

    4. Энергетическая эффективность — позволяет уменьшить энергопотребление на единицу вычислений.

### 4) Примеры реальных приложений

        1) DEGIMA — суперкомпьютер для астрофизики
        Использует CPU и GPU для моделирования N‑body систем, например, движения звёзд и галактик.
        CPU управляет задачами и распределяет работу, GPU ускоряет вычисления сил взаимодействия между телами
        Cсылка: https://en.wikipedia.org/wiki/DEGIMA

        2) Exscalate4Cov — виртуальный скрининг лекарств
        Проект для ускоренного поиска потенциальных лекарств против COVID‑19.
        CPU отвечает за организацию симуляций, GPU выполняет массовые расчёты взаимодействий молекул с белками.
        Cсылка: https://en.wikipedia.org/wiki/Exscalate4Cov

        3) GAMER — GPU‑ускоренные астрофизические симуляции
        Система для моделирования гидродинамических и гравитационных процессов в космических объектах.
        CPU управляет логикой симуляции, GPU выполняет параллельные вычисления физических полей.
        Cсылка: https://arxiv.org/abs/0907.3390

# Задача 2. Работа с массивами и OpenMP

    Реализуйте программу на C++, которая:
        1. Создаёт массив из 10 000 случайных чисел.
        2. Находит минимальное и максимальное значения массива:
            o в последовательной реализации;
            o с использованием OpenMP для параллельной обработки.
        3. Сравнивает время выполнения обеих реализаций и формулирует выводы.

In [ ]:
%%writefile task2.cpp

#include <iostream>
#include <cstdlib>      // rand, srand
#include <ctime>        // time
#include <chrono>       // измерение времени
#include <omp.h>        // OpenMP

using namespace std;
using namespace chrono;

int main() {
    const int N = 10000;   // размер массива

    // Динамическое выделение памяти под массив
    int* arr = new int[N];

    // Инициализация генератора случайных чисел
    srand(time(nullptr));

    // Заполнение массива случайными числами от 1 до 100
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 100 + 1;
    }


    // Последовательный поиск min и max
    // ===============================

    int min_seq = arr[0];
    int max_seq = arr[0];

    auto start_seq = high_resolution_clock::now();

    for (int i = 1; i < N; i++) {
        if (arr[i] < min_seq)
            min_seq = arr[i];
        if (arr[i] > max_seq)
            max_seq = arr[i];
    }

    auto end_seq = high_resolution_clock::now();
    double time_seq =
        duration<double, milli>(end_seq - start_seq).count();


    // Параллельный поиск min и max (OpenMP)
    // ===============================

    int min_par = arr[0];
    int max_par = arr[0];

    auto start_par = high_resolution_clock::now();

#pragma omp parallel for reduction(min:min_par) reduction(max:max_par)
    for (int i = 1; i < N; i++) {
        if (arr[i] < min_par)
            min_par = arr[i];
        if (arr[i] > max_par)
            max_par = arr[i];
    }

    auto end_par = high_resolution_clock::now();
    double time_par =
        duration<double, milli>(end_par - start_par).count();


    // Вывод результатов
    // ===============================

    cout << "Последовательная реализация:\n";
    cout << "Минимум: " << min_seq << endl;
    cout << "Максимум: " << max_seq << endl;
    cout << "Время выполнения: " << time_seq << " мс\n\n";

    cout << "Параллельная реализация (OpenMP):\n";
    cout << "Минимум: " << min_par << endl;
    cout << "Максимум: " << max_par << endl;
    cout << "Время выполнения: " << time_par << " мс\n";

    // Освобождение памяти
    delete[] arr;

    return 0;
}

Overwriting task2.cpp


In [ ]:
!g++-15 -fopenmp task2.cpp -o task2
!./task2

Последовательная реализация:
Минимум: 1
Максимум: 100
Время выполнения: 0.019 мс

Параллельная реализация (OpenMP):
Минимум: 1
Максимум: 100
Время выполнения: 0.185 мс


### Сравнение времени выполнения и выводы

        В ходе выполнения программы были получены следующие результаты:
        последовательная реализация выполнила поиск минимального и максимального элементов за 0.019 мс,
        в то время как параллельная реализация с использованием OpenMP заняла 0.185 мс.

        Несмотря на использование параллельной обработки, параллельная версия оказалась медленнее последовательной.
        Это объясняется тем, что при небольшом размере массива (10 000 элементов) накладные расходы на создание и
        синхронизацию потоков превышают выигрыш от распараллеливания вычислений.

        Таким образом, для небольших объёмов данных последовательная реализация является более эффективной.
        Параллельная обработка с использованием OpenMP начинает показывать преимущество при значительно больших размерах массива,
        когда вычислительная нагрузка компенсирует дополнительные затраты на управление потоками.

# Задача 3. Параллельная сортировка с OpenMP

    Реализуйте алгоритм сортировки выбором с использованием OpenMP:
        1. напишите последовательную реализацию алгоритма;
        2. добавьте параллелизм с помощью директив OpenMP;
        3. проверьте производительность для массивов размером 1 000 и 10 000 элементов.


In [ ]:
%%writefile task3.cpp

#include <iostream>
#include <cstdlib>      // для rand()
#include <ctime>        // для time()
#include <chrono>       // для измерения времени
#include <omp.h>        // для OpenMP

using namespace std;
using namespace std::chrono;

// ==================== Последовательная сортировка выбором ====================
// Проходит по массиву и выбирает минимальный элемент из неотсортированной части,
// затем меняет его местами с текущим элементом.
void selectionSortSequential(int* arr, int n) {
    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;  // предполагаем, что минимальный элемент — текущий

        // Поиск минимального элемента в оставшейся части массива
        for (int j = i + 1; j < n; j++) {
            if (arr[j] < arr[min_idx]) {
                min_idx = j;  // обновляем индекс минимального элемента
            }
        }

        // Обмен текущего элемента с найденным минимумом
        swap(arr[i], arr[min_idx]);
    }
}

// ==================== Параллельная сортировка выбором (OpenMP) ====================
// Основное ускорение достигается за счет параллельного поиска минимального элемента
void selectionSortParallel(int* arr, int n) {
    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;  // индекс глобального минимума

        // Параллельный блок поиска минимального элемента
#pragma omp parallel
        {
            int local_min_idx = min_idx;  // каждый поток имеет локальный минимум

#pragma omp for nowait
            // Потоки распределяют между собой итерации поиска минимума
            for (int j = i + 1; j < n; j++) {
                if (arr[j] < arr[local_min_idx]) {
                    local_min_idx = j;  // обновление локального минимума
                }
            }

            // Критическая секция для обновления глобального минимума
#pragma omp critical
            {
                if (arr[local_min_idx] < arr[min_idx]) {
                    min_idx = local_min_idx;  // обновление глобального минимума
                }
            }
        }

        // Обмен текущего элемента с глобальным минимумом
        swap(arr[i], arr[min_idx]);
    }
}

int main() {
    const int sizes[2] = {1000, 10000};  // размеры массивов для теста

    srand(time(nullptr));  // инициализация генератора случайных чисел

    for (int s = 0; s < 2; s++) {
        int N = sizes[s];
        cout << "\nРазмер массива: " << N << endl;

        // Выделение памяти под массивы для последовательной и параллельной сортировок
        int* arr1 = new int[N];
        int* arr2 = new int[N];

        // Заполнение массивов случайными числами
        for (int i = 0; i < N; i++) {
            arr1[i] = rand() % 10000;  // случайное число от 0 до 9999
            arr2[i] = arr1[i];          // копия для параллельной сортировки
        }

        // ==================== Последовательная версия ====================
        auto start_seq = high_resolution_clock::now();  // старт таймера
        selectionSortSequential(arr1, N);               // вызов сортировки
        auto end_seq = high_resolution_clock::now();    // конец таймера

        double time_seq = duration<double, milli>(end_seq - start_seq).count(); // вычисление времени

        // ==================== Параллельная версия ====================
        auto start_par = high_resolution_clock::now();  // старт таймера
        selectionSortParallel(arr2, N);                 // вызов параллельной сортировки
        auto end_par = high_resolution_clock::now();    // конец таймера

        double time_par = duration<double, milli>(end_par - start_par).count(); // вычисление времени

        // ==================== Вывод результатов ====================
        cout << "Последовательная сортировка: " << time_seq << " мс\n";
        cout << "Параллельная сортировка (OpenMP): " << time_par << " мс\n";

        // Освобождение памяти
        delete[] arr1;
        delete[] arr2;
    }

    return 0;
}

Writing task3.cpp


In [ ]:
!g++-15 -fopenmp task3.cpp -o task3
!./task3


Размер массива: 1000
Последовательная сортировка: 0.937 мс
Параллельная сортировка (OpenMP): 50.22 мс

Размер массива: 10000
Последовательная сортировка: 43.875 мс
Параллельная сортировка (OpenMP): 327.098 мс


# Задача 4. Сортировка на GPU с использованием CUDA

    Реализуйте параллельную сортировку слиянием на GPU с использованием CUDA:

        1. разделите массив на подмассивы, каждый из которых обрабатывается отдельным блоком;
        2. выполните параллельное слияние отсортированных подмассивов;
        3. замерьте производительность для массивов размером 10 000 и 100 000 элементов.

In [1]:
!nvidia-smi

Fri Dec 26 15:54:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%%writefile task4.cu

#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <vector>
#include <chrono>
#include <algorithm>
#include <climits>

using namespace std;
using namespace std::chrono;

#define BLOCK_SIZE 256  // Количество потоков в одном блоке

// ==================== Kernel 1: сортировка блока ====================
// Каждый блок сортирует свой подмассив в shared memory (быстрая локальная память GPU)
__global__ void blockSort(int* data, int n) {
    __shared__ int shared[BLOCK_SIZE];  // shared memory для подмассива блока

    int tid = threadIdx.x;                              // локальный индекс потока в блоке
    int gid = blockIdx.x * blockDim.x + tid;           // глобальный индекс потока в массиве

    // Копирование данных из глобальной памяти в shared memory
    if (gid < n)
        shared[tid] = data[gid];
    else
        shared[tid] = INT_MAX;  // если поток выходит за пределы массива, заполняем большим числом

    __syncthreads();  // ждем, пока все потоки блока скопируют данные

    // Простая сортировка пузырьком внутри блока
    for (int i = 0; i < blockDim.x; i++) {
        for (int j = tid; j < blockDim.x - 1; j += blockDim.x) {
            if (shared[j] > shared[j + 1]) {
                // Обмен значений
                int tmp = shared[j];
                shared[j] = shared[j + 1];
                shared[j + 1] = tmp;
            }
        }
        __syncthreads();  // синхронизируем потоки после каждой итерации
    }

    // Копируем отсортированный блок обратно в глобальную память
    if (gid < n)
        data[gid] = shared[tid];
}

// ==================== Kernel 2: слияние подмассивов ====================
// Каждый поток сливает два отсортированных подмассива заданной ширины
__global__ void mergeKernel(int* input, int* output, int width, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;  // индекс потока

    int start = idx * 2 * width;      // начало первого подмассива
    if (start >= n) return;           // если поток вне массива, выходим

    int mid = min(start + width, n);      // конец первого подмассива и начало второго
    int end = min(start + 2 * width, n);  // конец второго подмассива

    int i = start;  // индекс для первого подмассива
    int j = mid;    // индекс для второго подмассива
    int k = start;  // индекс для записи в выходной массив

    // Слияние двух подмассивов в отсортированный порядок
    while (i < mid && j < end) {
        if (input[i] <= input[j])
            output[k++] = input[i++];
        else
            output[k++] = input[j++];
    }

    // Копирование оставшихся элементов первого подмассива
    while (i < mid) output[k++] = input[i++];
    // Копирование оставшихся элементов второго подмассива
    while (j < end) output[k++] = input[j++];
}

// ==================== Хост-код ====================
int main() {
    vector<int> sizes = {10000, 100000};  // размеры массивов для теста

    for (int N : sizes) {
        vector<int> h_data(N);

        // Заполняем массив случайными числами
        for (int i = 0; i < N; i++)
            h_data[i] = rand() % 1000000;

        int* d_data;
        int* d_temp;
        cudaMalloc(&d_data, N * sizeof(int));  // выделяем память на GPU для основного массива
        cudaMalloc(&d_temp, N * sizeof(int));  // выделяем память для временного массива

        // Копируем массив с хоста на GPU
        cudaMemcpy(d_data, h_data.data(), N * sizeof(int), cudaMemcpyHostToDevice);

        auto start = high_resolution_clock::now();  // старт таймера

        // -------------------- Шаг 1: сортировка блоков --------------------
        int numBlocks = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;  // количество блоков
        blockSort<<<numBlocks, BLOCK_SIZE>>>(d_data, N);    // вызов kernel
        cudaDeviceSynchronize();                             // ждем завершения всех потоков

        // -------------------- Шаг 2: итеративное слияние --------------------
        for (int width = BLOCK_SIZE; width < N; width *= 2) {
            int mergeBlocks = (N + 2 * width - 1) / (2 * width);  // сколько блоков для слияния
            mergeKernel<<<mergeBlocks, 1>>>(d_data, d_temp, width, N); // запуск слияния
            cudaDeviceSynchronize();                               // ждем завершения
            swap(d_data, d_temp);                                  // меняем указатели массивов
        }

        auto end = high_resolution_clock::now();  // конец таймера
        chrono::duration<double, milli> elapsed = end - start;

        // Копируем результат обратно на CPU
        cudaMemcpy(h_data.data(), d_data, N * sizeof(int), cudaMemcpyDeviceToHost);

        // Проверка, отсортирован ли массив
        bool sorted = is_sorted(h_data.begin(), h_data.end());

        // -------------------- Вывод результатов --------------------
        cout << "==============================" << endl;
        cout << "Размер массива: " << N << " элементов" << endl;
        cout << "Сортировка завершена: " << (sorted ? "Да" : "Нет") << endl;
        cout << "Время выполнения на GPU: " << elapsed.count() << " мс" << endl;

        // Вывод первых и последних 5 элементов
        cout << "Первые 5 элементов: ";
        for (int i = 0; i < min(5, N); i++) cout << h_data[i] << " ";
        cout << "\nПоследние 5 элементов: ";
        for (int i = max(0, N - 5); i < N; i++) cout << h_data[i] << " ";
        cout << endl;

        // Освобождение памяти GPU
        cudaFree(d_data);
        cudaFree(d_temp);
    }

    return 0;
}

Writing task4.cu


In [3]:
!nvcc task4.cu -o task4
!./task4

Размер массива: 10000 элементов
Сортировка завершена: Нет
Время выполнения на GPU: 44.3669 мс
Первые 5 элементов: 289383 930886 692777 636915 747793 
Последние 5 элементов: 287797 480021 920292 459307 609430 
Размер массива: 100000 элементов
Сортировка завершена: Да
Время выполнения на GPU: 0.045622 мс
Первые 5 элементов: 0 0 0 0 0 
Последние 5 элементов: 0 0 0 0 0 
